In [9]:
import librosa as lb
import sounddevice as sd
import soundfile as sf
data, sr = lb.load('car1.m4a', sr=None)
sd.play(data)

C:\Users\ducha\AppData\Local\Temp\ipykernel_11272\3649671974.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sr = lb.load('car1.m4a', sr=None)
d:\tools\miniconda3\envs\dataml\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [11]:
car_raw = ['car1.m4a', 'car2.m4a', 'car3.m4a', 'car4.m4a', 'car5.m4a', 'car6.m4a', 'car7.m4a', 'car8.m4a']
tram_raw = ['tram1.m4a', 'tram2.m4a', 'tram3.m4a', 'tram4.m4a', 'tram5.m4a', 'tram6.m4a', 'tram7.m4a', 'tram8.m4a', 'tram9.m4a', 'tram10.m4a']
for car in car_raw:
    data, sr = lb.load(car, sr=None)
    sf.write(f'data/car/{car.replace("m4a", "wav")}', data, sr)
    print(data.shape)

for tram in tram_raw:
    data, sr = lb.load(tram, sr=None)
    sf.write(f'data/tram/{tram.replace("m4a", "wav")}', data, sr)
    print(data.shape)


C:\Users\ducha\AppData\Local\Temp\ipykernel_11272\1049630700.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sr = lb.load(car, sr=None)
d:\tools\miniconda3\envs\dataml\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


(488384,)
(373696,)
(345024,)
(193472,)
(168896,)
(361408,)
(385984,)
(320448,)


C:\Users\ducha\AppData\Local\Temp\ipykernel_11272\1049630700.py:9: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sr = lb.load(tram, sr=None)


(1024960,)
(1184704,)
(721856,)
(869312,)
(566208,)
(533440,)
(775104,)
(979904,)
(1065920,)
(1033152,)


In [13]:
data, sr = lb.load('data/car/car1.wav', sr=None)
sd.play(data)